## Insights from Visualization

- Train generator 1 -> Just not enough data to say anything
- Train generator 2 -> acce ts based 250ms cutline data. 553753 rows with only 10 sites, smallest wps count for site_floor is 3 with actual records 42, whereas the highest wps count is 835 with actual records 11724. It converges well with good accuracy. -> rmse_x: 0.61 rmse_y: 0.66 rmse_f: 0.017, comp metric at 0.67 -> But has no sites that are in the submission sites, so bad submission score
- Train generator 3 -> wps ts based with no cutline. 166681 rows with all 204 sites. It just doesn’t converge though -> rmse_x: 24.6 rmse_y: 21.8 rmse_f: 0.41 with comp metric at 30.1 -> Too large.
- Train generator 3M ->modify version 3 to only the 24 sites -> reduced to 75278 rows -> Still doesn’t converge. rmse_x: 23.8 rmse_y: 21.6 rmse_f: 0.36, comp metric at 29.6. -> Too large. Only wps maybe just isn't enough

- Train generator 2.1 -> acce_uncali ts based 100ms cutline data. Should cover all 204 site. Reduced to 100ms cutline because memory just won't hold. -> Need a way to convert 3.5G worth of pickle to datatable

In [ ]:
import os
import json
import glob
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil
import random

# Train data generation
import collections
import csv
from pathlib import Path
from typing import List, Tuple, Any

import time
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import pickle

import math

pd.set_option("display.max_columns", 100)

In [ ]:
# !cp -r ../input/indoorlocationgithub/* ./

In [ ]:
# # kaggle notebook
# from io_f import read_data_file
# from compute_f import compute_step_positions, compute_steps, \
# compute_headings, compute_stride_length, compute_step_heading, compute_rel_positions, split_ts_seq

#gcp
import compute_f
import io_f
import visualize_f
import main
from io_f import read_data_file
from compute_f import compute_step_positions, compute_steps, \
compute_headings, compute_stride_length, compute_step_heading, compute_rel_positions, split_ts_seq

In [ ]:
SEED = 42

In [ ]:
!ls -la -h ./

In [ ]:
# train2 had better score even if it covered only 10 sites
# create similar dataset to train2 but with all the sites
# but maybe we just only got 10 sites for a reason (no acce_uncali etc)

# train_file_name = "../input/indoorfulltestgroupedtrain/indoor_train_2_site_group.pkl"
# train_file_name = "../input/indoor2ndtrain/indoor_train_2.pkl"
# train_file_name = "../input/indoor3rdtrain/indoor_train_3.pkl"
train_file_name = "./indoor_train_4.pkl"
# test_file_name = "../input/indoorfulltestgroupedtrain/indoor_test_2_250ms.pkl"
# dir_path = "../input/indoorpkl/"

# Load data it back in
with open(train_file_name, "rb") as file:
    df_train = pickle.load(file)

# with open(test_file_name, "rb") as file:
#     df_test = pickle.load(file)

# Uncomment below for making train 3M
# Get submission file
sub_df = pd.read_csv("./input/sample_submission.csv")
sub_df[["site", "file", "timestamp"]] = sub_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))

# Filter train data by the site_id of submission file
test_site_ids = sub_df["site"].unique()
print(test_site_ids)
df_train = df_train[df_train["site_id"].isin(test_site_ids)]
print(len(df_train))
print(df_train["site_id"].value_counts())

In [ ]:
# # set the index for test data
# df_test = df_test.set_index("site_path_timestamp")
# display(df_test.head())

In [ ]:
# Train info
print("df len: ", len(df_train), "\n")
print("df column len: ", len(df_train.columns), "\n")
print("site_id nunique: ", df_train["site_id"].nunique(), "\n")
print("file_id nunique: ", df_train["file_id"].nunique(), "\n")
print("x nunique: ", df_train["x"].nunique(), "\n")
print("y nunique: ", df_train["y"].nunique(), "\n")
print("event ts nunique: ", df_train["ts"].nunique(), "\n")
print("start ts nunique: ", df_train["start_ts"].nunique(), "\n") # should be same length as file_id
print("diff_ts_wp_ts nunique: ", df_train["diff_ts_wp_ts"].nunique(), "\n")

In [ ]:
# # Test info
# print("df len: ", len(df_test), "\n")
# print("df column len: ", len(df_test.columns), "\n")
# print("site_id nunique: ", df_test["site_id"].nunique(), "\n")
# print("file_id nunique: ", df_test["file_id"].nunique(), "\n")
# print("x nunique: ", df_test["x"].nunique(), "\n")
# print("y nunique: ", df_test["y"].nunique(), "\n")
# print("event ts nunique: ", df_test["ts"].nunique(), "\n")
# print("start ts nunique: ", df_test["start_ts"].nunique(), "\n") # should be same length as file_id
# print("diff_ts_wp_ts nunique: ", df_test["diff_ts_wp_ts"].nunique(), "\n")

In [ ]:
# test_site_id = df_test["site_id"].unique()
# train_site_id = df_train["site_id"].unique()
# print(test_site_id, "\n")
# print(train_site_id, "\n")
# a = list(set(test_site_id) & set(train_site_id))
# print(len(a))

In [ ]:
# # print("test_df site_id value_counts: ", df_test["file_id"].value_counts(), "\n")
# # print("sub_df site_id value_counts: ", sub_df["file"].value_counts(), "\n")

# # Matching sites: 5d2709d403f801723c32bd39, 5dc8cea7659e181adb076a3f, 5d2709b303f801723c327472
# # Missing sites: 5dbc1d84c1eb61796cf7c010, 5da958dd46f8266d0737457b, 5d2709bb03f801723c32852c, 5d27096c03f801723c31e5e0

# # Matching files: f4bd7479dbe91f3d18f44c49, 4affd104e0ec7a7806edfa78, 610f2c07b26508790d1cd355
# # Missing files: d72ceac4628436a540910d98, 2b4bacedc942ffcb523ff20f, 876cd27fc8f63a2800fc3de8
# base_path = "../input/indoor-location-navigation/test/"
# match_path = "4affd104e0ec7a7806edfa78"
# missing_path = "d72ceac4628436a540910d98"

# match_path_full = base_path + match_path + ".txt"
# missing_path_full = base_path + missing_path + ".txt"

# #timestamps in sub_df
# sub_df_a = sub_df[sub_df["file"] == match_path]
# sub_df_a_ts = sorted(sub_df_a["timestamp"].value_counts().keys())
# print(sub_df_a_ts[0])
# df_test_match = df_test[(df_test["file_id"] == match_path) & (df_test["closest_wp_ts"] == int(sub_df_a_ts[0]))]
# display(df_test_match.head())

# sub_df_b = sub_df[sub_df["file"] == missing_path]
# sub_df_b_ts = sorted(sub_df_b["timestamp"].value_counts().keys())
# # display(sub_df_b.head())

# # ts for submission files
# print(sub_df_a_ts[:5], "\n")
# print(sub_df_b_ts[:5])

In [ ]:
# metric
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt((xhat - x)**2 + (yhat - y)**2) + 15 * np.abs(fhat-f)
    return intermediate.sum() / xhat.shape[0]

In [ ]:
print(len(df_train.columns))
df_train = df_train.dropna(axis=1, how="all")
df_train = df_train.dropna(axis=0, how="any")
print(len(df_train.columns))

null_cols = df_train.columns[df_train.isna().any()].tolist()
print("Columns with nulls: ", null_cols)

def fill_na_mean(df, cols, groupby_col):
    for col in cols:
        df[col] = df.groupby(groupby_col).transform(lambda x: x.fillna(x.mean()))

# 4 min for 100,000 rows
# fill_na_mean(df_train, tqdm(null_cols), "site_id")

# null_cols = df_train.columns[df_train.isna().any()].tolist()
# print("Check if nulls are filled: ", null_cols)
print("Current columns: ", df_train.columns)

In [ ]:
# column settings

# latter is to drop all the ts related columns
# drop_cols = ["floor_converted", "floor", "x", "y", "floor_le", "file_id", "file_id_le", "diff_start_wp_ts"]
# drop_cols = ["floor_converted", "floor", "x", "y", "floor_le", "file_id", "file_id_le", \
#              "ts", "start_ts", "diff_start_ts", "closest_wp_ts", "diff_start_wp_ts", "diff_ts_wp_ts", \
#              "acce_ts", "diff_acce_ts", "ahrs_ts", "diff_ahrs_ts", "magn_ts", "diff_magn_ts", \
#              "gyro_ts", "diff_gyro_ts", "acce_u_ts", "diff_acce_u_ts", "magn_u_ts", "diff_magn_u_ts", \
#              "gyro_u_ts", "diff_gyro_u_ts", "wifi_ts", "diff_wifi_ts", "wifi_last_seen_ts", \
#              "beacon_ts", "diff_beacon_ts", "rel_ts", "diff_rel_ts", "ts_date", "ts_day", \
#              "ts_hour", "ts_minute", "wifi_last_seen_ts_date", "wifi_last_seen_ts_day", \
#              "wifi_last_seen_ts_hour", "wifi_last_seen_ts_minute"
#             ]

# categorical_features = ["site_id", "within_500ms", "within_1000ms", "wifi_ssid", \
#                         "wifi_bssid", "beacon_ssid"] # train 2
# categorical_features = ["site_id", "wifi_ssid", "wifi_bssid", "beacon_ssid"] # train 3
categorical_features = ["site_id", "within_100ms", "within_200ms", \
                        "wifi_ssid", "wifi_bssid", "beacon_ssid"] # train 4

int_features = ["floor_converted", "acce_u_ts", "diff_acce_u_ts", "acce_u_x", "acce_u_y", "acce_u_z",\
                "gyro_u_ts", "diff_gyro_u_ts", "gyro_u_x", "gyro_u_y", "gyro_u_z",\
                "wifi_rssi", "wifi_ts", "diff_wifi_ts", "wifi_last_seen_ts", 
                "beacon_rssi", "beacon_ts", "diff_beacon_ts", \
                "ts_date", "ts_day", "ts_hour", "ts_minute", \
                "wifi_last_seen_ts_date", "wifi_last_seen_ts_day", \
                "wifi_last_seen_ts_hour", "wifi_last_seen_ts_minute"]

# convert to category from object dtype
for col in categorical_features:
    df_train[col] = df_train[col].astype("category")

# convert to int from datetime64 dtype
for col in int_features:
    df_train[col] = df_train[col].astype(int)

In [ ]:
# print(df_train.dtypes[:50])
display(df_train.head())

In [ ]:
df_train.info()

In [ ]:
# dataset prep for training
target_x = df_train.iloc[:, 7]
target_y = df_train.iloc[:, 8]
target_f = df_train.iloc[:, 2]

# drop target columns
# drop_cols = ["file_id", "floor_converted", "floor", "x", "y"] # Train 2 test
drop_cols = ["file_id", "file_id_le", "floor_converted", "floor", "x", "y", "floor_le"] # Train 3
# drop_cols = ["file_id", "file_id_le", "floor_converted", "floor", \
#              "floor_converted_le", "x", "y"] # Train 2
# drop_cols = ["file_id", "file_id_le", "floor_converted", "floor", "floor_converted_le", "x", "y"]
features = df_train.drop(columns=drop_cols)
display(features.head())

# set target cols
targets = ["x", "y", "f"]
target_data = [target_x, target_y, target_f]

d = {}
for tgt, tgt_data in zip(targets, target_data):
    feat_train, feat_val, target_train, target_val = \
    train_test_split(features, tgt_data, test_size = 0.2, random_state = SEED)
    feat_train, feat_viz, target_train, target_viz = \
    train_test_split(feat_train, target_train, test_size = 0.2, random_state = SEED)
    d["feat_train_{}".format(tgt)] = feat_train
    d["feat_val_{}".format(tgt)] = feat_val
    d["feat_viz_{}".format(tgt)] = feat_viz
    d["target_train_{}".format(tgt)] = target_train
    d["target_val_{}".format(tgt)] = target_val
    d["target_viz_{}".format(tgt)] = target_viz
    d["train_{}".format(tgt)] = lgb.Dataset(data=feat_train, label=target_train, categorical_feature=categorical_features, free_raw_data=False).construct()
    d["val_{}".format(tgt)] = lgb.Dataset(data=feat_val, label=target_val, categorical_feature=categorical_features, free_raw_data=False).construct()
    print(d["feat_train_{}".format(tgt)].shape)
    print(d["feat_val_{}".format(tgt)].shape)
    print(d["feat_viz_{}".format(tgt)].shape)
    print(d["target_train_{}".format(tgt)].shape)
    print(d["target_val_{}".format(tgt)].shape)
    print(d["target_viz_{}".format(tgt)].shape)
    print(d["train_{}".format(tgt)].get_data().shape)
    print(d["train_{}".format(tgt)].get_label().shape)
    print(d["val_{}".format(tgt)].get_data().shape)
    print(d["val_{}".format(tgt)].get_label().shape)

In [ ]:
# lgb patams
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000, # example had 50000
              'learning_rate': 0.1,
              'num_leaves': 50,
              'min_data_in_leaf': 50,
              'max_depth': 15,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

# train models
start = time.time()

for tgt in targets:
    model = lgb.train(params=lgb_params,
                      train_set=d["train_{}".format(tgt)],
                      early_stopping_rounds=20,
                      valid_sets=d["val_{}".format(tgt)])
    d["model_{}".format(tgt)] = model
    d["pred_target_{}".format(tgt)] = model.predict(d["feat_val_{}".format(tgt)], 
                                                    num_iteration=model.best_iteration)

In [ ]:
print("time to process one train_path", time.time() - start)
print(d.keys())
print(len(d["target_val_f"]))
print(len(d["pred_target_f"]))
# print(np.argwhere(np.isnan(list(d["target_val_f"]))))

# the below code should not be there once the model has proper floor_map
d["target_val_f"] = [0 if math.isnan(x) else x for x in d["target_val_f"]]

In [ ]:
mse_x = mean_squared_error(d["target_val_x"], d["pred_target_x"])
mse_y = mean_squared_error(d["target_val_y"], d["pred_target_y"])
mse_f = mean_squared_error(d["target_val_f"], d["pred_target_f"])
rmse_x = np.sqrt(mse_x)
rmse_y = np.sqrt(mse_y)
rmse_f = np.sqrt(mse_f)
print("rmse_x:", rmse_x, "rmse_y:",rmse_y, "rmse_f:",rmse_f)

In [ ]:
target_val_x = d["target_val_x"].to_numpy()
target_val_y = d["target_val_y"].to_numpy()
target_val_f = np.array(d["target_val_f"]).astype(float)
#target_val_f = target_val_f
print(type(target_val_x[0]))
print(type(target_val_y[0]))
print(type(target_val_f[0]))

comp_metric = comp_metric(d["pred_target_x"], d["pred_target_y"], d["pred_target_f"],
                          target_val_x, target_val_y, target_val_f)

print("comp metric: ", comp_metric)

In [ ]:
lgb.plot_importance(d["model_x"], max_num_features=20, grid=False, figsize=(16,8))
lgb.plot_importance(d["model_y"], max_num_features=20, grid=False, figsize=(16,8))
lgb.plot_importance(d["model_f"], max_num_features=20, grid=False, figsize=(16,8))

In [ ]:
# pickle.dump(d["model_x"], open("x_model_train_4.pkl", "wb"))
# pickle.dump(d["model_y"], open("y_model_train_4.pkl", "wb"))
# pickle.dump(d["model_f"], open("f_model_train_4.pkl", "wb"))

---
## Use the trained model to predict viz data
---

In [ ]:
# Run prediction against train data
print(d["feat_viz_x"].shape)
print(d["feat_viz_y"].shape)
print(d["target_viz_x"].shape)

In [ ]:
# Make predictions using the viz data
modelx = d["model_x"]
modely = d["model_y"]
modelf = d["model_f"]

# Make predictions with test data
viz_predsx = modelx.predict(d["feat_viz_x"])
viz_predsy = modely.predict(d["feat_viz_y"])
viz_predsf = modelf.predict(d["feat_viz_f"])

In [ ]:
# quick check
# overlap_check = set(d["feat_viz_x"].index) ^ set(d["feat_viz_f"].index)
# print(overlap_check)

df_viz = d["feat_viz_x"]
df_viz["x"] = d["target_viz_x"]
df_viz["y"] = d["target_viz_y"]
df_viz["f"] = d["target_viz_f"]
df_viz["x_pred"] = viz_predsx
df_viz["y_pred"] = viz_predsy
df_viz["f_pred"] = viz_predsf
df_viz["x_diff"] = abs(df_viz["x"] - df_viz["x_pred"])
df_viz["y_diff"] = abs(df_viz["y"] - df_viz["y_pred"])
df_viz["f_diff"] = abs(df_viz["f"] - df_viz["f_pred"])

# create mix of site_id_le x floor
df_viz["site_le_f"] = "site_" + df_viz["site_id_le"].astype(str) + "_floor_" + df_viz["f"].astype(str)

# add difference measurement based on the competition's metric
df_viz["metric"] = np.sqrt((df_viz["x"] - df_viz["x_pred"])**2 + (df_viz["y"] - df_viz["y_pred"])**2) + (15 * abs(df_viz["f"] - df_viz["f_pred"]))

display(df_viz.head())
print(len(viz_predsx))

In [ ]:
# set sites with most/mid/least counts
site_le_f_counts = df_viz["site_le_f"].value_counts()
site_most_count = site_le_f_counts.idxmax()
site_least_count = site_le_f_counts.idxmin()
site_mid_count = int(len(site_le_f_counts) / 2)
site_count_list = df_viz["site_le_f"].value_counts().index.tolist()

print("site_floor with most count: ", site_most_count, "->", site_le_f_counts[site_most_count])
print("site_floor with mid count: ", site_count_list[site_mid_count], "->", site_le_f_counts[site_count_list[site_mid_count]])
print("site_floor with least count: ", site_least_count, "->", site_le_f_counts[site_least_count])

In [ ]:
x_max = df_viz["x"].max()
x_min = df_viz["x"].min()
y_max = df_viz["y"].max()
y_min = df_viz["y"].min()

# Plot predictions against ground truths
def plot_gt_pred(site, scale=True):
    df_viz_a = df_viz[df_viz["site_le_f"] == site] # most counts
    x = df_viz_a["x"]
    y = df_viz_a["y"]
    x_pred = df_viz_a["x_pred"]
    y_pred = df_viz_a["y_pred"]

    fig = plt.figure(figsize=(8,8))
    fig.patch.set_facecolor("white")
    ax1 = fig.add_subplot(111)
    ax1.scatter(x_pred, y_pred, s=10, c="g", marker="o", label="predicted", alpha=0.3)
    ax1.scatter(x, y, s=10, c="r", marker="s", label="true values")
    ax1.scatter(x, y, s=10, c="r", marker="s", label="true values")
    if scale == True:
        ax1.set_xlim([x_min, x_max])
        ax1.set_ylim([y_min, y_max])
    ax1.set_title(site)
    plt.legend(loc='upper left');
    plt.show()

plot_gt_pred(site_most_count, scale=False)
plot_gt_pred(site_count_list[site_mid_count], scale=False)
plot_gt_pred(site_least_count, scale=False)

# plot_gt_pred(site_most_count, scale=True)
# plot_gt_pred(site_count_list[site_mid_count], scale=True)
# plot_gt_pred(site_least_count, scale=True)

In [ ]:
# Take difference of truth - predicted -> which has the largest difference
display(df_viz.head())
print(len(df_viz))
print(df_viz["ts"].nunique())

# # train 2
# df_viz_grouped = df_viz.groupby("site_le_f").agg({"site_le_f": "count", \
#                                                   "closest_wp_ts": lambda x: x.nunique(), \
#                                                   "x_diff": "mean", "y_diff": "mean", \
#                                                   "f_diff": "mean", "metric": "mean"
#                                                  })
# df_viz_grouped = df_viz_grouped.sort_values(by=["closest_wp_ts"], ascending=False)

# train 3
df_viz_grouped = df_viz.groupby("site_le_f").agg({"site_le_f": "count", \
                                                  "ts": lambda x: x.nunique(), \
                                                  "x_diff": "mean", "y_diff": "mean", \
                                                  "f_diff": "mean", "metric": "mean"
                                                 })
df_viz_grouped = df_viz_grouped.sort_values(by=["ts"], ascending=False)
print(len(df_viz_grouped))
display(df_viz_grouped)

In [ ]:
supposed_comp_metric = df_viz_grouped["metric"].mean()
print("supposed_comp_metric: ", supposed_comp_metric)

# Visualize count and comp metric
a = df_viz_grouped["site_le_f"]
b = df_viz_grouped["metric"]

fig = plt.figure(figsize=(10,10))
fig.patch.set_facecolor("white")
ax1 = fig.add_subplot(111)
ax1.scatter(a, b, s=10, c="r", marker="s", label="Count x Metric mean")
plt.legend(loc='upper left');
plt.show()

In [ ]:
# Try seeing other sites with high metric mean
metric_higest = df_viz_grouped["metric"].idxmax()
metric_lowest = df_viz_grouped["metric"].idxmin()

plot_gt_pred(metric_higest, scale=False)
plot_gt_pred(metric_lowest, scale=False)

In [ ]:
# Plot ground truth and predictions on actual map
# Low metric mean -> waypoints are scattered, and intervals seem sparce
# Set scales right for all graphs

---
## Making Quick Prediction
---

In [ ]:
# Visualize differences between test 2 and test 3
# !ls -la -h ../input/indoorfulltestgroupedtrain

In [ ]:
# test_2 = "../input/indoorfulltestgroupedtrain/indoor_test_2_250ms.pkl"
test_3 = "./indoor_test_3.pkl"

# with open(test_2, "rb") as file:
#     df_test_2 = pickle.load(file)

with open(test_3, "rb") as file:
    df_test_3 = pickle.load(file)

### Test Visualization: Diff timestamp distribution

In [ ]:
# # Check if either is missing some data that should be there
# test2_ts_id = df_test_2["site_path_timestamp"].unique()
# test3_ts_id = df_test_3["site_path_timestamp"].unique()
# print("len of test2_ts_id", len(test2_ts_id))
# print("len of test3_ts_id", len(test3_ts_id), "\n")
# no_overlap_ts = set(test2_ts_id) ^ set(test3_ts_id)
# print("not overlapping ts", no_overlap_ts)

# # Check columns are the same
# print(len(df_test_2))
# print(len(df_test_3))
# no_overlap_col = set(df_test_2.columns) ^ set(df_test_3.columns)
# print("not overlapping columns", no_overlap_col)

# # Check the diff timestamps -> check if we're not picking up something that are just too far away from ts
# def print_minmax(df, cols):
#     for col in cols:
#         print("col: ", col)
#         print("max: ", df[col].max())
#         print("min: ", df[col].min())

# check_diff_cols = ["diff_acce_ts", "diff_ahrs_ts", "diff_magn_ts", \
#                    "diff_gyro_ts", "diff_acce_u_ts", "diff_magn_u_ts", \
#                    "diff_gyro_u_ts", "diff_wifi_ts", "diff_beacon_ts", \
#                    "diff_rel_ts"]

# print("~~~~~~~~~~~~~~~~~~~~~~~~")
# print("test 2 range: ")
# print_minmax(df_test_2, check_diff_cols)
# print("~~~~~~~~~~~~~~~~~~~~~~~~", "\n")

# print("~~~~~~~~~~~~~~~~~~~~~~~~")
# print("test 3 range: ")
# print_minmax(df_test_3, check_diff_cols)

In [ ]:
# def bin_plot(df, col, title, x_label, y_label):
#     min_v = pd.to_numeric(df[col]).min()
#     max_v = pd.to_numeric(df[col]).max()
#     diff = max_v - min_v
#     fig = plt.figure(figsize=(16.0, 4.0))
#     fig.patch.set_facecolor("white")
#     ax = fig.add_subplot(1,1,1)
#     ax.hist(df[col], bins=30)
#     ax.set_title(title)
#     # ax.set_xlabel(x_label)
#     ax.text(0.01, 0.99, "min is {}, max is {}, diff is {}".format(min_v, max_v, diff), verticalalignment='top', transform=ax.transAxes)
#     ax.set_ylabel(y_label)
#     plt.xticks(rotation=90)
#     fig.show()


# check_diff_cols = ["diff_acce_ts", "diff_ahrs_ts", "diff_magn_ts", \
#                    "diff_gyro_ts", "diff_acce_u_ts", "diff_magn_u_ts", \
#                    "diff_gyro_u_ts", "diff_wifi_ts", "diff_beacon_ts", \
#                    "diff_rel_ts"]

# df_test_2.name = "df_test_2"
# df_test_3.name = "df_test_3"

# for df in [df_test_2, df_test_3]:
#     for col in check_diff_cols:
#         df_title = "df_test_2" if df.name == "df_test_2" else "df_test_3"
#         bin_plot(df, col, f"{df_title} --- {col}", f"{col}", "freq")

In [ ]:
# # Prune the outliers -> For acce etc @ ±200ms, for wifi&rel @ ±2500ms, for beacon @
# # print(len(df_test_3))
# a = df_test_3[abs(df_test_3["diff_acce_ts"]) > 200]
# b = df_test_3[abs(df_test_3["diff_wifi_ts"]) > 2500]
# c = df_test_3[abs(df_test_3["diff_beacon_ts"]) > 10000]
# d = df_test_3[abs(df_test_3["diff_rel_ts"]) > 2500]
# print(df_test_3["diff_acce_ts"].mean())
# print(df_test_3["diff_wifi_ts"].mean())
# print(df_test_3["diff_beacon_ts"].mean())
# print(df_test_3["diff_rel_ts"].mean())
# print("\n")
# print(df_test_3["diff_acce_ts"].std())
# print(df_test_3["diff_wifi_ts"].std())
# print(df_test_3["diff_beacon_ts"].std())
# print(df_test_3["diff_rel_ts"].std())
# print("\n")
# print(len(a))
# print(len(b))
# print(len(c))
# print(len(d))
# df_test_3.info()
# # df_test_3["diff_beacon_ts"]
# # df_test_3["diff_rel_ts"]

### ↑Til here, so don't delete these cells

In [ ]:
display(df_test_3.head())
display(features.head())

In [ ]:
# df_test_3.info()

In [ ]:
# Train 4 & Test 3 Specific stuff

# Drop any column that is not in the features

# Fill "within" columns with True
df_test_3["within_250ms"] = df_test_3["within_250ms"].fillna(True)
df_test_3["within_500ms"] = df_test_3["within_500ms"].fillna(True)
df_test_3 = df_test_3.rename({"within_250ms": "within_100ms", "within_500ms": "within_200ms"}, axis=1)

# Drop the columns with all null values
df_test_3 = df_test_3.dropna(axis=1, how='all')

# get the column names that are still in the df_test but not features
no_test_col_overlap = set(features.columns) ^ set(df_test_3.columns)
# a = list(set(features.columns) & set(test_features.columns))
print(no_test_col_overlap)

categorical_features = ["site_id", "within_100ms", "within_200ms", \
                        "wifi_ssid", "wifi_bssid", "beacon_ssid"]
int_features = ["ts_date", "ts_day", "ts_hour", "ts_minute", \
                "wifi_last_seen_ts_date", "wifi_last_seen_ts_day", \
                "wifi_last_seen_ts_hour", "wifi_last_seen_ts_minute"]

# convert to category from object dtype
for col in categorical_features:
    df_test_3[col] = df_test_3[col].astype("category")

# convert to int from datetime64 dtype
for col in int_features:
    df_test_3[col] = df_test_3[col].astype(int)

# drop cols
drop_cols = ['floor_le', 'file_id', 'file_id_le']
df_test_3 = df_test_3.drop(columns=drop_cols)
test_features = df_test_3.set_index("site_path_timestamp")

In [ ]:
no_test_col_overlap = set(features.columns) ^ set(df_test_3.columns)
# a = list(set(features.columns) & set(test_features.columns))
print(no_test_col_overlap)

In [ ]:
# # Train 3/3M & Test 3 Specific stuff

# # Drop any column that is not in the features

# # Drop the columns with all null values
# df_test_3 = df_test_3.dropna(axis=1, how='all')

# # get the column names that are still in the df_test but not features
# no_test_col_overlap = set(features.columns) ^ set(df_test_3.columns)
# # a = list(set(features.columns) & set(test_features.columns))
# print(no_test_col_overlap)

# categorical_features = ["site_id", "wifi_ssid", "wifi_bssid", "beacon_ssid"]
# int_features = ["ts_date", "ts_day", "ts_hour", "ts_minute", \
#                 "wifi_last_seen_ts_date", "wifi_last_seen_ts_day", \
#                 "wifi_last_seen_ts_hour", "wifi_last_seen_ts_minute"]

# # convert to category from object dtype
# for col in categorical_features:
#     df_test_3[col] = df_test_3[col].astype("category")

# # convert to int from datetime64 dtype
# for col in int_features:
#     df_test_3[col] = df_test_3[col].astype(int)

# # drop cols
# drop_cols = ['file_id_le', 'diff_ts_wp_ts', 'diff_start_wp_ts', 'floor_le', 'closest_wp_ts', 'file_id']
# df_test_3 = df_test_3.drop(columns=drop_cols)
# test_features = df_test_3.set_index("site_path_timestamp")

In [ ]:
# # Train 2 & Test 3 Specific stuff

# # Drop any column that is not in the features

# # Fill "within" columns with True
# df_test_3["within_250ms"] = df_test_3["within_250ms"].fillna(True)
# df_test_3["within_500ms"] = df_test_3["within_500ms"].fillna(True)
# df_test_3 = df_test_3.rename({"within_250ms": "within_500ms", "within_500ms": "within_1000ms"}, axis=1)

# # Drop the columns with all null values
# df_test_3 = df_test_3.dropna(axis=1, how='all')

# # get the column names that are still in the df_test but not features
# no_test_col_overlap = set(features.columns) ^ set(df_test_3.columns)
# # a = list(set(features.columns) & set(test_features.columns))
# print(no_test_col_overlap)

# categorical_features = ["site_id", "within_500ms", "within_1000ms", "wifi_ssid", "wifi_bssid", "beacon_ssid"]
# int_features = ["ts_date", "ts_day", "ts_hour", "ts_minute", \
#                 "wifi_last_seen_ts_date", "wifi_last_seen_ts_day", \
#                 "wifi_last_seen_ts_hour", "wifi_last_seen_ts_minute"]

# # convert to category from object dtype
# for col in categorical_features:
#     df_test_3[col] = df_test_3[col].astype("category")

# # convert to int from datetime64 dtype
# for col in int_features:
#     df_test_3[col] = df_test_3[col].astype(int)

# # drop cols
# drop_cols = ['file_id_le', 'floor_le', 'file_id']
# df_test_3 = df_test_3.drop(columns=drop_cols)
# test_features = df_test_3.set_index("site_path_timestamp")

In [ ]:
# # print(len(df_test_3.columns))
# # # df_test = df_test.dropna(axis=1, how='all')
# # df_test_3 = df_test_3.dropna(axis=1, how='all')
# # df_test_3 = df_test_3.dropna(axis=0, how='any')
# # print(len(df_test.columns))

# null_cols = df_test_3.columns[df_test_3.isna().any()].tolist()
# print("Columns with nulls: ", null_cols)

# # def fill_na_mean(df, cols, groupby_col):
# #     for col in cols:
# #         df[col] = df.groupby(groupby_col).transform(lambda x: x.fillna(x.mean()))

# # 4 min for 100,000 rows
# # fill_na_mean(df_test, tqdm(null_cols), "site_id")

# # null_cols = df_test.columns[df_test.isna().any()].tolist()
# # print("Check if nulls are filled: ", null_cols)


# # column settings
# # rename within_250ms and within_500ms first
# # df_test = df_test.rename({"within_250ms": "within_100ms", "within_500ms": "within_200ms"}, axis=1)

# # latter is to drop all the ts related columns
# # drop_cols = ["floor_converted", "floor", "x", "y", "floor_le", "file_id", "file_id_le", "diff_start_wp_ts"]
# # drop_cols = ["floor_converted", "floor", "x", "y", "floor_le", "file_id", "file_id_le", \
# #              "ts", "start_ts", "diff_start_ts", "closest_wp_ts", "diff_start_wp_ts", "diff_ts_wp_ts", \
# #              "acce_ts", "diff_acce_ts", "ahrs_ts", "diff_ahrs_ts", "magn_ts", "diff_magn_ts", \
# #              "gyro_ts", "diff_gyro_ts", "acce_u_ts", "diff_acce_u_ts", "magn_u_ts", "diff_magn_u_ts", \
# #              "gyro_u_ts", "diff_gyro_u_ts", "wifi_ts", "diff_wifi_ts", "wifi_last_seen_ts", \
# #              "beacon_ts", "diff_beacon_ts", "rel_ts", "diff_rel_ts", "ts_date", "ts_day", \
# #              "ts_hour", "ts_minute", "wifi_last_seen_ts_date", "wifi_last_seen_ts_day", \
# #              "wifi_last_seen_ts_hour", "wifi_last_seen_ts_minute"
# #             ]

# # categorical_features = ["site_id", "within_100ms", "within_200ms", "wifi_ssid", "wifi_bssid", "beacon_ssid"]
# categorical_features = ["site_id", "within_250ms", "within_500ms", "wifi_ssid", "wifi_bssid", "beacon_ssid"]
# int_features = ["acce_u_ts", "diff_acce_u_ts", "acce_u_x", "acce_u_y", "acce_u_z",\
#                 "gyro_u_ts", "diff_gyro_u_ts", "gyro_u_x", "gyro_u_y", "gyro_u_z",\
#                 "wifi_rssi", "wifi_ts", "diff_wifi_ts", "wifi_last_seen_ts", 
#                 "beacon_rssi", "beacon_ts", "diff_beacon_ts", \
#                 "ts_date", "ts_day", "ts_hour", "ts_minute", \
#                 "wifi_last_seen_ts_date", "wifi_last_seen_ts_day", \
#                 "wifi_last_seen_ts_hour", "wifi_last_seen_ts_minute"]

# # convert to category from object dtype
# for col in categorical_features:
#     df_test[col] = df_test[col].astype("category")

# # convert to int from datetime64 dtype
# for col in int_features:
#     df_test[col] = df_test[col].astype(int)

# # drop cols
# drop_cols = ["file_id", "floor_le"]
# df_test = df_test.drop(columns=drop_cols)
# test_features = df_test.set_index("site_path_timestamp")

In [ ]:
print(len(df_test_3))
print("site_id nunique: ", df_test_3["site_id"].nunique(), "\n")
display(test_features.head())
print(len(features.columns))
print(len(test_features.columns))
no_overlap = set(features.columns) ^ set(test_features.columns)
print(no_overlap)

In [ ]:
# test_features.info()
# features.info()

In [ ]:
# Load models
modelx = d["model_x"]
modely = d["model_y"]
modelf = d["model_f"]

# Make predictions with test data
test_predsx = modelx.predict(test_features)
test_predsy = modely.predict(test_features)
test_predsf = modelf.predict(test_features)

In [ ]:
df_preds = pd.DataFrame(np.stack((test_features.index, test_predsf, test_predsx, test_predsy))).T
df_preds.columns = ["site_path_timestamp", "floor", "x", "y"]
# test_preds = test_preds.set_index("site_path_timestamp")
print("df_preds before groupby: ")
display(df_preds.head())

# groupby to take the average
# convert to float from object
for col in ["floor", "x", "y"]:
    df_preds[col] = df_preds[col].astype(float)

df_pred_grouped = df_preds.groupby(["site_path_timestamp"]).mean()
df_pred_grouped = df_pred_grouped.reset_index()

# round floor value to integer
df_pred_grouped["floor"] = df_pred_grouped["floor"].round(decimals=0)

print("len of df_pred_grouped: ", len(df_pred_grouped)) # should be same as sub_df length, but not yet
print("df_pred_grouped after groupby: ")
display(df_pred_grouped.head())

# find the missing site_path_timestamp and fill it with average
# get submission file
sub_df = pd.read_csv("./input/sample_submission.csv")
print("sub_df: ")
display(sub_df.head())

subdf_site_id = sub_df["site_path_timestamp"].unique()
pred_site_id = df_pred_grouped["site_path_timestamp"].unique()
print("len of subdf_site_id", len(subdf_site_id), "\n")
print("len of pred_site_id", len(pred_site_id), "\n")
no_overlap = set(subdf_site_id) ^ set(pred_site_id)
print("no overlap", no_overlap)

# missing timestamps -> fill in with average value for now
# '5dbc1d84c1eb61796cf7c010_67bf4b03fc22542bb6d51daf_0000000000011'
# '5d2709e003f801723c32d896_e3455e53350336857caf08f0_0000000119903'
floor_avg = round(df_pred_grouped["floor"].mean())
x_avg = df_pred_grouped["x"].mean()
y_avg = df_pred_grouped["y"].mean()
print("avg values for floor, x, y", floor_avg, x_avg, y_avg)

for i, ts in enumerate(list(no_overlap)):
    row = [ts, floor_avg, x_avg, y_avg]
    row_num = len(df_pred_grouped)
    df_pred_grouped.loc[row_num + i] = row

print("len of pred_grouped", len(df_pred_grouped))
print("tail of df_pred_grouped to check missing two rows are appended: ")
display(df_pred_grouped.tail())

In [ ]:
# csv for submission
df_pred_grouped.to_csv("submission.csv", index=False)